#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

C:\Users\mat\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000.0,2011-08-17 06:31:31,1145.0,canceled,24,US,1145.0,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000.0,2012-07-23 20:46:48,1851.0,failed,28,US,1851.0,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000.0,2016-07-01 21:55:54,7534.0,successful,254,US,3796.0,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000.0,2017-08-28 15:47:02,16298.0,successful,367,US,2670.0,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000.0,2016-01-25 17:37:10,1.0,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [5]:
df_ks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                150000 non-null  int64  
 1   name              149998 non-null  object 
 2   category          150000 non-null  object 
 3   main_category     150000 non-null  object 
 4   currency          150000 non-null  object 
 5   deadline          150000 non-null  object 
 6   goal              150000 non-null  object 
 7   launched          150000 non-null  object 
 8   pledged           150000 non-null  object 
 9   state             150000 non-null  object 
 10  backers           150000 non-null  object 
 11  country           150000 non-null  object 
 12  usd pledged       148518 non-null  object 
 13  usd_pledged_real  150000 non-null  float64
dtypes: float64(1), int64(1), object(12)
memory usage: 16.0+ MB


In [6]:
df_ks['launched'] = pd.to_datetime(df_ks['launched'], errors='coerce')
df_ks = df_ks.dropna(subset = ['launched'])
df_ks = df_ks.rename(columns = {"ID": "_id"})

In [7]:
df_ks = df_ks.rename({"usd pledged":"usd_pledged"}, axis='columns')
df_ks['goal'] = pd.to_numeric(df_ks['goal'],errors = 'coerce')
df_ks['pledged'] = pd.to_numeric(df_ks['pledged'], errors = 'coerce')
df_ks['usd_pledged'] = pd.to_numeric(df_ks['usd_pledged'],errors = 'coerce')

### Importer les données

In [9]:
df_ks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149999 entries, 0 to 149999
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   _id               149999 non-null  int64         
 1   name              149997 non-null  object        
 2   category          149999 non-null  object        
 3   main_category     149999 non-null  object        
 4   currency          149999 non-null  object        
 5   deadline          149999 non-null  object        
 6   goal              149999 non-null  float64       
 7   launched          149999 non-null  datetime64[ns]
 8   pledged           149999 non-null  float64       
 9   state             149999 non-null  object        
 10  backers           149999 non-null  object        
 11  country           149999 non-null  object        
 12  usd_pledged       148517 non-null  float64       
 13  usd_pledged_real  149999 non-null  float64       
dtypes: d

In [10]:
collection.delete_many({})
collection.insert_many(df_ks.to_dict(orient='records'))

## Question 1  

In [11]:
dons = collection.find().sort([("pledged",-1)]).limit(5)
pd.DataFrame(dons)

,_id,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,usd_pledged_real
0,342886736,COOLEST COOLER: 21st Century Cooler that's Act...,Product Design,Design,USD,2014-08-30,50000.0,2014-07-08 10:14:37,13285226.36,successful,62642,US,1.328523e+07,1.328523e+07
1,2103598555,"Pebble 2, Time 2 + All-New Pebble Core",Product Design,Design,USD,2016-06-30,1000000.0,2016-05-24 15:49:52,12779843.49,successful,66673,US,1.277984e+07,1.277984e+07
2,2111201788,Expect the Unexpected. digiFilmï¿½ Camera by Y...,Product Design,Design,HKD,2017-11-19,800000.0,2017-10-10 12:52:25,10035296.00,successful,6935,HK,4.788642e+03,1.285143e+06
3,1033978702,OUYA: A New Kind of Video Game Console,Gaming Hardware,Games,USD,2012-08-09,950000.0,2012-07-10 14:44:41,8596474.58,successful,63416,US,8.596475e+06,8.596475e+06
4,450099426,"The Everyday Backpack, Tote, and Sling",Product Design,Design,USD,2016-09-10,500000.0,2016-07-13 00:47:35,6565782.50,successful,26359,US,1.462611e+06,6.565782e+06


## Question 2

In [12]:
success = collection.count_documents({'state' : 'successful'})
print(str(success) + " projets ont atteint leur but")

53040 projets ont atteint leur but


## Question 3

In [13]:
projet_category= collection.aggregate([{"$group" : {"_id" : "$category","Nombre de projet" : {"$sum" : 1}}}])
pd.DataFrame(projet_category)

,_id,Nombre de projet
0,Jazz,733
1,Robots,237
2,Space Exploration,137
3,Faith,439
4,Zines,144
...,...,...
154,Fine Art,291
155,Spaces,331
156,Workshops,59
157,Electronic Music,858


## Question 4

In [14]:
import datetime
year = datetime.datetime.strptime("2016-01-01",'%Y-%m-%d')
collection.count_documents({"$and" :[{"launched" : {"$lt" : year}},{"country" : "FR"}]})

330

## Question 5

In [15]:
cursor = collection.find({"$and":[{"usd_pledged_real":{"$gte": 200000}}, {"country":"US"}]})
pd.DataFrame(cursor)

,_id,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,usd_pledged_real
0,217543389,The uKeg Pressurized Growler for Fresh Beer,Drinks,Food,USD,2014-12-08,75000.0,2014-10-15 06:34:48,1559525.68,successful,10293,US,1559525.68,1559525.68
1,909248984,Redux COURG - Hybrid Watches with Missions to ...,Product Design,Design,USD,2015-08-20,30000.0,2015-07-21 19:01:41,692912.00,successful,2200,US,692912.00,692912.00
2,1688905333,Legion Solar - A Better Way to Energy Independ...,Technology,Technology,USD,2016-06-18,50000.0,2016-05-03 16:00:22,277524.00,successful,399,US,277524.00,277524.00
3,447408232,Edyn: Welcome to the connected garden.,Hardware,Technology,USD,2014-07-08,100000.0,2014-06-05 13:50:13,384201.63,successful,2336,US,384201.63,384201.63
4,1919184836,Period Panties,Fashion,Fashion,USD,2014-03-09,10000.0,2014-02-07 15:32:50,404763.30,successful,9550,US,404763.30,404763.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,1270104074,Republique by Camouflaj + Logan,Video Games,Games,USD,2012-05-12,500000.0,2012-04-10 14:37:30,555662.86,successful,11611,US,555662.86,555662.86
654,344582158,Hub+ for USB-C: Get your MacBook ports back.,Hardware,Technology,USD,2015-06-16,35000.0,2015-05-05 19:08:49,883460.80,successful,8019,US,883460.80,883460.80
655,170672682,"The Field Skillet: Lighter, Smoother Cast Iron",Food,Food,USD,2016-04-07,30000.0,2016-03-07 02:25:16,1633361.53,successful,12553,US,1633361.53,1633361.53
656,78391894,Handcrafted Dress Shoes Reinvented for the Mod...,Footwear,Fashion,USD,2017-04-27,20000.0,2017-03-14 12:43:55,1265430.42,successful,4422,US,339955.15,1265430.42


## Question 6 

In [16]:
from bson.regex import Regex
sport_name = collection.count_documents({"name" : Regex("Sport")})
print(str(sport_name)+" projets ont sport dans leur nom")

323 projets ont sport dans leur nom
